In [ ]:
# ────────────────────────────────────────────────────────────────
# 03_interpretation_association.ipynb
# Visualize posteriors + overlay known events
# ────────────────────────────────────────────────────────────────

import pandas as pd
import numpy as np
import arviz as az
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Load data & trace (run after 02_change_point_model)
df = pd.read_csv("../data/processed/brent_model_ready.csv", parse_dates=["Date"])
trace = az.from_netcdf("trace_single.nc")   # ← save trace earlier or re-run model

# ─── Load events ────────────────────────────────────────────────────
events = pd.read_csv("../data/external/events.csv", parse_dates=["Date"])

# ─── Price plot with change point uncertainty ───────────────────────
plt.figure(figsize=(15, 6))

# Original price
plt.plot(df["Date"], df["price"], lw=0.9, color="navy", alpha=0.8, label="Brent price")

# Mean before / after from posterior
mu1_mean = trace.posterior["mu_1"].mean().item()
mu2_mean = trace.posterior["mu_2"].mean().item()
tau_mean = trace.posterior["tau"].mean().item().astype(int)

plt.axhline(mu1_mean, xmin=0, xmax=tau_mean/len(df), color="C0", ls="--", alpha=0.7, label=f"μ before ≈ ${mu1_mean:.0f}")
plt.axhline(mu2_mean, xmin=tau_mean/len(df), xmax=1, color="C3", ls="--", alpha=0.7, label=f"μ after ≈ ${mu2_mean:.0f}")

# Change point posterior density (on second axis)
ax2 = plt.twinx()
tau_samples = trace.posterior["tau"].values.flatten()
ax2.hist(df["Date"].iloc[tau_samples], bins=60, density=True, color="teal", alpha=0.35, label="Posterior P(τ)")
ax2.set_ylabel("Change point posterior density", color="teal")

# Plot known events
for _, row in events.iterrows():
    dt = row["Date"]
    if dt < df["Date"].min() or dt > df["Date"].max():
        continue
    plt.axvline(dt, color="darkred", alpha=0.4, ls=":", lw=1.4)
    plt.text(dt, df["price"].max()*0.92, row["Event_Description"], rotation=90,
             va="top", ha="right", fontsize=8.5,
             bbox=dict(facecolor="white", alpha=0.85, edgecolor="none"))

plt.title("Brent price + detected change point + major events", fontsize=14)
plt.ylabel("USD / barrel")
plt.grid(alpha=0.3)

ax2.grid(False)
lines1, labels1 = plt.gca().get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
plt.legend(lines1 + lines2, labels1 + labels2, loc="upper left", fontsize=9)

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=6))
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

# Simple impact statement
change_pct = 100 * (mu2_mean - mu1_mean) / mu1_mean
print(f"Estimated mean change: {change_pct:+.1f}%  (${mu1_mean:.0f} → ${mu2_mean:.0f})")

In [ ]:
import pandas as pd

tau_map = trace.posterior["tau"].mode(dim=["chain","draw"]).item()
date_map = df["Date"].iloc[tau_map]

results = pd.DataFrame({
    "Change Point Index": [tau_map],
    "Date": [date_map.date()],
    "Mean Before": [trace.posterior["mu_before"].mean().item()],
    "Mean After":  [trace.posterior["mu_after"].mean().item()],
    "% Change":    [100 * (trace.posterior["mu_after"].mean() - trace.posterior["mu_before"].mean()) / trace.posterior["mu_before"].mean()]
})

print(results.round(2))
results.to_csv("../reports/change_point_summary.csv", index=False)